<a href="https://colab.research.google.com/github/sayed-kassem/NN_Torch/blob/main/Seq2Seq/NMT_ENG_FR_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [60]:
SOS_token = 0
EOS_token = 1

class Lang:
  def __init__(self,name):
    self.name = name
    self.word2index={}
    self.word2count={}
    self.index2word= {0: "SOS", 1:"EOS"}
    self.n_words = 2

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self,word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words +=1
    else:
      self.word2count[word] +=1

def unicodeToAscii(s):
    return ''.join( c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    #low, trim , remove non-letter characters
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1",s)
    s = re.sub(r"[^a-zA-z!?]+",r" ",s)
    return s.strip()

def readLangs(lang1, lang2, reverse=False):
  print("Reading Lines...")
  #Read file and split in to lines
  lines = open('data/%s-%s.txt' %(lang1,lang2), encoding='utf-8').read().strip().split('\n')
  #Split every line into pairs and normalize
  pairs = [[normalizeString(s) for s in l.split('\t')] for l  in lines]
  #Reverse pairs, make Lang Instances
  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
  else:
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
  return input_lang, output_lang, pairs

In [61]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
  return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading Lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je manque d espace dans mon placard', 'i m running out of closet space']


In [62]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size,dropout_p=0.1):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size,hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first = True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, input):
    embedded = self.dropout(self.embedding(input))
    output, hidden = self.gru(embedded)
    return output, hidden

class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.embedding = nn.Embedding(output_size,hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size,batch_first= True)
    self.out = nn.Linear(hidden_size, output_size)

  def forward (self, encoder_outputs, encoder_hidden, target_tensor=None):
    batch_size = encoder_outputs.size(0)
    decoder_input = torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
    decoder_hidden = encoder_hidden
    decoder_outputs = []

    for i in range(MAX_LENGTH):
      decoder_output, decoder_hidden = self.forward_step(decoder_input,decoder_hidden)
      decoder_outputs.append(decoder_output)

      if target_tensor is not None:
        decoder_input = target_tensor[:,i].unsqueeze(1)
      else:
        _, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze(-1).detach()

      decoder_outputs = torch.cat(decoder_outputs, dim=1)
      decoder_outputs = F.log_softmax(decoder_outputs, dim=1)
      return decoder_outputs, decoder_hidden, None

    def forward_step(self,input, hidden):
      output = self.embedding(input)
      output= F.relu(output)
      output, hidden = self.gru(output,hidden)
      output = self.out(output)
      return output, hidden

In [63]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

#Preparing Training Data

In [64]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [65]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [66]:
import time
import math

def asMinute(s):
  m = math.floor(s/60)
  s-=m*60
  return "%dm %ds" %(m,s)
def timeSince(since, percent):
  now = time.time()
  s = now - since
  es = s/(percent)
  rs = es - s
  return "%s (- %s)" %(asMinute(s), asMinute(rs))


In [67]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [68]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [69]:
def evaluate(encoder,decoder,sentence,input_lang,output_lang):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang,sentence)

    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs,encoder_hidden)

    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()

    decoded_words = []
    for idx in decoded_ids:
      if idx.item() == EOS_token:
        decoded_words.append("<EOS>")
        break
      decoded_words.append(output_lang.index2word[idx.item()])
  return decoded_words, decoder_attn


In [70]:
def evaluateRandomly(encoder,decoder, n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, _ = evaluate(encoder,decoder,pair[0], input_lang, output_lang)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print(' ')


hidden_size = 128
batch_size =32
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
encoder = EncoderRNN(input_lang.n_words,hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words). to(device)

train(train_dataloader,encoder,decoder,100, print_every=10, plot_every=10)

Reading Lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
1m 12s (- 10m 49s) (10 10%) 1.0917
2m 24s (- 9m 36s) (20 20%) 0.2652
3m 36s (- 8m 24s) (30 30%) 0.1009
4m 48s (- 7m 12s) (40 40%) 0.0583
6m 0s (- 6m 0s) (50 50%) 0.0429
7m 12s (- 4m 48s) (60 60%) 0.0359
8m 24s (- 3m 36s) (70 70%) 0.0320
9m 36s (- 2m 24s) (80 80%) 0.0298
10m 48s (- 1m 12s) (90 90%) 0.0280
12m 1s (- 0m 0s) (100 100%) 0.0269


In [71]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> elle est plus vieille que lui
= she s older than him
< she s older than him <EOS>
 
> je suis heureuse pour vous deux
= i m happy for you both
< i m happy for you both <EOS>
 
> il n est pas aussi grand que son frere
= he is not as tall as his brother
< he is not as tall as his brother <EOS>
 
> tu trouves toujours des reproches a me faire
= you re always finding fault with me
< you re always finding fault with me <EOS>
 
> tu es trop faible
= you re too weak
< you re too weak <EOS>
 
> c est une personne tres sympa
= she is a very nice person
< she is a very nice person <EOS>
 
> je suis celibataire
= i m single
< i am single <EOS>
 
> nous allons voyager en voiture
= we are going to travel by car
< we are going to travel by car <EOS>
 
> vous etes toutes malades
= you re all insane
< you re all insane <EOS>
 
> je suis heureux d accepter votre invitation
= i am glad to accept your invitation
< i am glad to accept your invitation <EOS>
 


In [72]:
def evaluateTestSentence(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))


evaluateTestSentence('il n est pas aussi grand que son pere')

evaluateTestSentence('je suis trop fatigue pour conduire')

evaluateTestSentence('je suis desole si c est une question idiote')

evaluateTestSentence('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive <EOS>
input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>
